# Investigating Factors Affecting Housing Rental Yields (Using ONS Data)

## Purpose

The aim of this notebook is to use various public ONS (`https://www.ons.gov.uk/`) datasets to find factors affecting housing yields. I hope this will give readers (and potential investors) a better idea of what drives rental yields, hopefully allowing for better decision making.

Note: Here I look at yields instead of raw house prices since yields are comparable across different locations.

I approximate average yields using `median(rental income)/median(house price)` since the actual average yield, `median(rental income / house price)`, requires data for each house and is otherwise unavailable.


## Data Used

Below I list the ONS datasets I will be using. The full urls are given at the end.

- Crime
- Private Rental Statistics
- Median House Price
- Mean House Price
- Property Sales
- Population Estimates
- Quality of Life Estimates

These are the datasets I found on the ONS website I thought will be relevant and suitable to work with.

These give data reported by local authority so we will have a reasonable number of data points (roughly 300) to perform inference on.

In [49]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from ons_processing import parse_files

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
csv_dir = "./ons_processing/data"

In [52]:
datasets = ["property_sales", "crime", "rental_summary", "population_age", "wellbeing", "property_prices"]
data = {k: getattr(parse_files, f"parse_{k}")(csv_dir) for k in datasets}
{k: df.shape for k, df in data.items()}

/home/nikul/Documents/Programming/ons_housing/ons_processing/parse_files.py:21: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(csv_path, **read_kwargs)


{'property_sales': (336, 2),
 'crime': (311, 4),
 'rental_summary': (357, 4),
 'population_age': (331, 6),
 'wellbeing': (421, 5),
 'property_prices': (331, 2)}

## Data Sources

Crime
- https://www.ons.gov.uk/peoplepopulationandcommunity/crimeandjustice/datasets/recordedcrimedatabycommunitysafetypartnershiparea

Private Rental Statistics
- https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/privaterentalmarketsummarystatisticsinengland

Median House Price
- https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/medianpricepaidbywardhpssadataset37

Mean House Price
- https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/meanpricepaidbywardhpssadataset38

Property Sales
- https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/numberofresidentialpropertysalesbywardhpssadataset36

Population Estimates
- https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/wardlevelmidyearpopulationestimatesexperimental

Quality of Life Estimates
- https://www.ons.gov.uk/datasets/wellbeing-local-authority/editions/time-series/versions/2

Note: the ONS has an API (https://developer.ons.gov.uk/) however these not all of these datasets are not available through this at the time of writing. Also, due issues parsing these excel files, I have manually exported the data to csvs which can be straightforwardly parsed.